<a href="https://colab.research.google.com/github/Ataerman/LLM_RAG-AI_AGENT/blob/main/Agent_with_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-openai langchain-cohere langchain-community \
sentence-transformers faiss-cpu \
datasets accelerate \
tiktoken \
cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/14

In [ ]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
openai_api_key=userdata.get('openai')
cohere_api_key=userdata.get('cohere')
openai_llm=ChatOpenAI(model='gpt-3.5-turbo', #creating llm model (using chatgpt 3.5 with api)
                      openai_api_key=openai_api_key,
                      temperature=0.3)

In [ ]:
text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics.
Since its premiere, Interstellar gained a cult following, and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

doc = Document(page_content=text)
# 🔹 Text Splitter: Break the text into smaller chunks with overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = splitter.split_documents([doc])
print(docs[0].page_content)

# 🔹 Embedding Model: Convert chunks into numerical vector representations
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 🔹 FAISS Vectorstore: Store and index the embeddings for similarity search
vectorstore = FAISS.from_documents(docs, embeddings)

Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher


<ipython-input-5-8ff98dd1a149>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.chains import RetrievalQA

# 🔹 Retriever: First retrieve top 10 similar chunks with FAISS
# 🔹 Then rerank them using Cohere Rerank API to improve relevance
retriever = ContextualCompressionRetriever(
    base_compressor=CohereRerank(
        cohere_api_key=cohere_api_key,
        model='rerank-multilingual-v2.0',
        top_n=3  # Return top 3 reranked chunks
    ),
    base_retriever=vectorstore.as_retriever(search_kwargs={'k': 10})
)
qa_chain=RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    chain_type="stuff")

In [ ]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.6 MB/s eta 0:00:00


In [ ]:
from langchain.agents import create_react_agent,AgentExecutor,load_tools
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import Tool
from langchain.tools.ddg_search.tool import DuckDuckGoSearchResults
from langchain.prompts import ChatPromptTemplate
rag_tool=Tool(
    name='Rag-Tool',
    description='Useful for answering questions about Interstellar',
    func=qa_chain.invoke)
search=DuckDuckGoSearchResults()
search_tool=Tool(
    name='duckduck',
    description='A web search engine. Use this to answer general knowledge and real-time queries.',
    func=search.run)
tools=load_tools(['llm-math'],llm=openai_llm)
tools+=[rag_tool,search_tool]

memory=ConversationBufferWindowMemory(
    memory_key='chat_history',
    return_messages=True,
    k=10)

<ipython-input-9-398b3d742f96>:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferWindowMemory(


In [ ]:
from langchain import PromptTemplate
#Create ReAct Template
react_template = """
Answer the following questions as best you can. You have access to the following tools:
{tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
...(this Thought/Action/Action Input/Observation can repeat N times)
Thought: I know the final answer
Final Answer: the final answer to the original input question

Previous conversation:
{chat_history}

Begin!

Question: {input}
{agent_scratchpad}
"""

prompt=PromptTemplate(template=react_template,
                      input_variables=['tools','tool_names','input','agent_scratchpad'])


agent=create_react_agent(openai_llm,tools,prompt)
agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)


In [ ]:
response = agent_executor.invoke({"input": "Who was the scientific consultant for the movie Interstellar?"})
print(response["output"])




> Entering new AgentExecutor chain...
Thought: This is a question about the movie Interstellar, so I should use the Rag-Tool to find the answer.
Action: Rag-Tool
Action Input: {"input": "Who was the scientific consultant for the movie Interstellar?"}{'query': '{"input": "Who was the scientific consultant for the movie Interstellar?"}', 'result': 'The scientific consultant for the movie Interstellar was Kip Thorne.'}Final Answer: The scientific consultant for the movie Interstellar was Kip Thorne.

> Finished chain.
The scientific consultant for the movie Interstellar was Kip Thorne.


In [ ]:
response = agent_executor.invoke({"input": "What is the square root of 144 multiplied by 8?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: This is a math question that involves finding the square root of a number and then multiplying it by another number.
Action: Calculator
Action Input: Calculate square root of 144 * 8Answer: 33.94112549695428Final Answer: The square root of 144 multiplied by 8 is approximately 33.94.

> Finished chain.
The square root of 144 multiplied by 8 is approximately 33.94.


In [ ]:
response = agent_executor.invoke({"input": "Who won the Best Actor Oscar in 2024?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: I need to search for the winner of the Best Actor Oscar in 2024.
Action: duckduck
Action Input: "Best Actor Oscar winner 2024"snippet: At the 2024 Oscars, Cillian Murphy won the best actor Oscar for his role of physicist J. Robert Oppenheimer, known as "the father of the atomic bomb.", title: Who won the best actor Oscar last year? See list of recent winners, link: https://www.usatoday.com/story/entertainment/movies/2025/02/19/oscar-best-actor-2024/78472302007/, snippet: The Academy Award for Best Actor is an award presented annually by the Academy of Motion Picture Arts and Sciences (AMPAS). It has been awarded since the 1st Academy Awards to an actor who has delivered an outstanding performance in a leading role in a film released that year. The award is traditionally presented by the previous year's Best Actress winner. However, in recent years, it has ..., title: Academy Award for Best Actor - Wikipedia, link: https://en.wikipedia.or

In [ ]:
response = agent_executor.invoke({"input": "What was the budget of Interstellar?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: This question requires information about the budget of the movie Interstellar.
Action: Use duckduck to search for the budget of the movie Interstellar.
Action Input: tool_input = "budget of Interstellar"Use duckduck to search for the budget of the movie Interstellar. is not a valid tool, try one of [Calculator, Rag-Tool, duckduck].Action: Use duckduck to search for the budget of the movie Interstellar.
Action Input: tool_input = "budget of Interstellar"Use duckduck to search for the budget of the movie Interstellar. is not a valid tool, try one of [Calculator, Rag-Tool, duckduck].Final Answer: Unfortunately, we were unable to retrieve the budget of the movie Interstellar using the available tools.

> Finished chain.
Unfortunately, we were unable to retrieve the budget of the movie Interstellar using the available tools.


In [ ]:
response = agent_executor.invoke({"input": "What was the my question about math ?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To find out the question about math, we can use the information provided in the previous conversation.
Action: Rag-Tool
Action Input: {"input": "What was the budget of Interstellar?"}{'query': '{"input": "What was the budget of Interstellar?"}', 'result': "I don't know."}Action: Rag-Tool
Action Input: {"input": "What is the square root of 144 multiplied by 8?"}{'query': '{"input": "What is the square root of 144 multiplied by 8?"}', 'result': 'The square root of 144 is 12. Multiplying 12 by 8 gives you 96.'}Final Answer: The question about math was "What is the square root of 144 multiplied by 8?"

> Finished chain.
The question about math was "What is the square root of 144 multiplied by 8?"
